In [11]:
import minsearch
import json

In [12]:
with open('documents.json', 'rt') as data_file:
    docs_raw = json.load(data_file)

In [13]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [14]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [15]:
index = minsearch.Index(text_fields=['question',"text", "section"], 
               keyword_fields=["course"])

In [16]:
index.fit(documents)

In [17]:
q = 'The course has started, can i still enroll?'

In [18]:
boost ={'question':3.0,'section':0.5}

results = index.search(
    query = q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost, num_results=5
    
)

In [19]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [20]:
from openai import OpenAI

In [21]:
client = OpenAI()

In [22]:
response = client.chat.completions.create(model ='gpt-4o', messages =[{"role":"user", "content":q}])

In [23]:
response.choices[0].message.content

"Whether you can still enroll in a course after it has started depends on several factors, such as the policies of the institution offering the course, the specific course structure, and how far along the course is. Here are some steps you can take:\n\n1. **Check the Course or University Policies:** Look up the specific enrollment policies of the institution or platform offering the course. Some institutions have a grace period during which you can still enroll in courses even after they have started.\n\n2. **Contact the Instructor:** If the course policies are not clear, reach out directly to the course instructor or the administrative office. Explain your situation and ask if it is still possible to enroll.\n\n3. **Review Course Requirements:** If the course has only just started, it's often more feasible to catch up on any missed material. However, if it’s already several weeks in, consider whether you will be able to catch up on the coursework without compromising your learning exp

In [64]:
def search(query):
    boost ={'question':3.0,'section':0.5}

    results = index.search(
        query = query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost, num_results=5
        
    )
    return results

In [65]:
def build_prompt(query,search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT form the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
  
    
    QUESTION:{question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section:{doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    prompt=prompt_template.format(question=query, context=context).strip()
    return prompt

In [66]:
def llm(prompt):
    response = client.chat.completions.create(model ='gpt-4o', messages =[{"role":"user", "content":prompt}])
    return response.choices[0].message.content
    

In [70]:
query = 'how do I run kafka?'

def rag(query):
    results = search(query)
    prompt = build_prompt(query,results)
    answer = llm(prompt)
    return answer

In [71]:
rag(query)

'To run Kafka, it depends on whether you are working with Java or Python. Below are the steps for both:\n\n### Java Kafka:\n1. Navigate to the project directory.\n2. Run the following command in the terminal:\n   ```\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\n### Python Kafka:\n1. Create a virtual environment and install the necessary packages:\n   ```sh\n   python -m venv env\n   source env/bin/activate\n   pip install -r ../requirements.txt\n   ```\n2. To activate the virtual environment each time you need it:\n   ```sh\n   source env/bin/activate\n   ```\n3. To deactivate the virtual environment:\n   ```sh\n   deactivate\n   ```\n   *Note: On Windows, use `env/Scripts/activate` instead of `env/bin/activate`.*\n\nAdditionally, ensure Docker images are up and running before you start running Python files.\n\nIf you encounter a permission issue with `./build.sh`, run:\n```sh\nchmod +x build.sh\n```'